<a href="https://colab.research.google.com/github/Rajnish379/Machine_Learning/blob/main/Weka_versus_my_knn_rmalred1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn import datasets

# Load the Iris dataset
iris = datasets.load_iris()
iris_data = iris.data
# iris_target contains the encoded integer which indicates the target class of the specific row i.e, 0 for setosa, 1 for versicolor and 2 for virginica.
iris_target = iris.target

In [ ]:
# We are ignoring the third column i.e., column with index 2 from the iris dataset
X_train = iris_data[::2,[0, 1, 3]] # Slicing for elements with even indices starting from 0 with a step of 2
X_test = iris_data[1::2,[0,1,3]] # Slicing for elements with odd indices starting from 1 with a step of 2
# X is capital because X_train is a matrix which contains n number of of rows and m number of columns

In [ ]:
# iris.target_names contains the decoded class name for the iris flower species (setosa for 0, versicolor for 1 and virginica for 2)
# We need to decode the class names especially because we have to create arff files for weka. Weka uses actual class names instead of 0,1 and 2 which is why we have to do this
y_train = iris.target_names[iris_target[::2]] # We are storing the decoded names from the target of iris dataset corresponding to even indices in y_train
y_test = iris.target_names[iris_target[1::2]] # We are storing the decoded names from the target of iris dataset corresponding to odd indices in y_test
# Here y is small in y_train because it usually indicates that each row of it has only one element

In [ ]:
feature_names = iris.feature_names[:]
print(feature_names)
del feature_names[2] # We don't need the third feature header so deleting it
feature_names = [x.replace(' ','') for x in feature_names]
print(feature_names)

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['sepallength(cm)', 'sepalwidth(cm)', 'petalwidth(cm)']


In [ ]:
# In concatenate, axis = 1 means horizontal concatenation and 0 is for vertical
column_names = feature_names + ['Class-labels']
Xy_train = np.hstack((X_train, y_train.reshape(-1,1))) # Here y_train and y_test are reshaped such that it has only one column instead of one row as before
Xy_test = np.hstack((X_test,y_test.reshape(-1,1)))
train_matrix = np.vstack((column_names, Xy_train))
test_matrix = np.vstack((column_names,Xy_test))

In [ ]:
np.savetxt('training_set.csv', train_matrix, '%s', delimiter=',')
np.savetxt('test_set.csv',test_matrix,'%s',delimiter = ',')

In [ ]:
# Function to calculate the distance between each row of two datasets
def calculate_distances(train_data, test_data):
    distances = np.zeros((test_data.shape[0], train_data.shape[0]))

    for i, test_row in enumerate(test_data):
        for j, train_row in enumerate(train_data):
            # Euclidean distance
#            distances[i, j] = np.linalg.norm(test_row - train_row)
            distances[i, j] = np.sqrt(np.sum((test_row - train_row)**2))

    return distances

In [ ]:
n_test = len(X_test)

nearest_neighbors_indices = np.zeros(n_test, dtype=int)

for i in range(n_test):
    # Calculate distances from the i-th sample to all samples in the training set
    distances = calculate_distances(X_train, X_test[i].reshape(1, -1)).flatten() #should return 1-by-75 distances
    distances[i] = np.inf

    nearest_neighbors_indices[i] = np.argmin(distances)
nearest_neighbors_indices

array([17, 15,  8,  0, 17, 12, 19,  8, 20, 23, 22, 13, 17, 14,  1, 10,  7,
       15,  2,  0,  4, 13, 15,  1, 17, 28, 40, 48, 49, 45, 39, 39, 43, 41,
       40, 67, 67, 29, 43, 46, 45, 39, 35, 34, 45, 39, 49, 48, 37, 48, 71,
       63, 61, 65, 60, 64, 71, 52, 60, 34, 71, 63, 51, 69, 26, 61, 27, 61,
       58, 56, 60, 60, 70, 58, 69])

In [ ]:
preds = y_train[nearest_neighbors_indices]

In [ ]:
sum(preds == y_test)

70

In [ ]:
(preds == y_test).mean()

0.9333333333333333

In [ ]:
np.where(preds != y_test)

(array([35, 36, 59, 64, 66]),)

(array([35, 36, 59, 64, 66]),) These data points present in the training set are misclassified. The training set consists of 75 rows which are labeled starting from 0 to 74.